In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import sem
from scipy.stats import linregress

In [2]:
#get data

file2019 = "399417274_T_T100_MARKET_ALL_CARRIER.csv"
file2020 = "399417274_T_T100_MARKET_ALL_CARRIER2.csv"

data2019 = pd.read_csv(file2019)
data2020 = pd.read_csv(file2020)

df_list = [data2019,data2020]

alldata = pd.concat(df_list)
alldata.columns

Index(['PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'UNIQUE_CARRIER',
       'AIRLINE_ID', 'UNIQUE_CARRIER_NAME', 'UNIQUE_CARRIER_ENTITY', 'REGION',
       'CARRIER', 'CARRIER_NAME', 'CARRIER_GROUP', 'CARRIER_GROUP_NEW',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP', 'CLASS',
       'DATA_SOURCE', 'Unnamed: 41'],
      dtype='object')

In [3]:
# set list of passenger flight classes
flightclass = ['A','C','E','F']

passdata = alldata.loc[(alldata['MONTH']<8)&((alldata['CLASS'] =='A') | (alldata['CLASS'] =='C') | (alldata['CLASS'] =='E') | (alldata['CLASS'] =='F'))  ,:]

In [4]:
passdata = passdata[['UNIQUE_CARRIER_NAME', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP','PASSENGERS', 'DISTANCE', 'CLASS']]
passdata.head()

,UNIQUE_CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST,DEST_CITY_NAME,...,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,YEAR,QUARTER,MONTH,DISTANCE_GROUP,PASSENGERS,DISTANCE,CLASS
0,Southwest Airlines Co.,PBI,"West Palm Beach/Palm Beach, FL",FL,Florida,US,United States,33,DAL,"Dallas, TX",...,US,United States,74,2019,2,6,3,0.0,1091.0,F
1,Southwest Airlines Co.,PHL,"Philadelphia, PA",PA,Pennsylvania,US,United States,23,GEG,"Spokane, WA",...,US,United States,93,2019,2,6,5,0.0,2155.0,F
2,Southwest Airlines Co.,PVD,"Providence, RI",RI,Rhode Island,US,United States,15,ATL,"Atlanta, GA",...,US,United States,34,2019,2,6,2,0.0,903.0,F
3,Southwest Airlines Co.,RIC,"Richmond, VA",VA,Virginia,US,United States,38,DEN,"Denver, CO",...,US,United States,82,2019,2,6,3,0.0,1482.0,F
4,Southwest Airlines Co.,RIC,"Richmond, VA",VA,Virginia,US,United States,38,LAS,"Las Vegas, NV",...,US,United States,85,2019,2,6,5,0.0,2086.0,F


In [5]:
passdata['flight'] = passdata['ORIGIN']+passdata['DEST']
passdata.head(20)

,UNIQUE_CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST,DEST_CITY_NAME,...,DEST_COUNTRY_NAME,DEST_WAC,YEAR,QUARTER,MONTH,DISTANCE_GROUP,PASSENGERS,DISTANCE,CLASS,flight
0,Southwest Airlines Co.,PBI,"West Palm Beach/Palm Beach, FL",FL,Florida,US,United States,33,DAL,"Dallas, TX",...,United States,74,2019,2,6,3,0.0,1091.0,F,PBIDAL
1,Southwest Airlines Co.,PHL,"Philadelphia, PA",PA,Pennsylvania,US,United States,23,GEG,"Spokane, WA",...,United States,93,2019,2,6,5,0.0,2155.0,F,PHLGEG
2,Southwest Airlines Co.,PVD,"Providence, RI",RI,Rhode Island,US,United States,15,ATL,"Atlanta, GA",...,United States,34,2019,2,6,2,0.0,903.0,F,PVDATL
3,Southwest Airlines Co.,RIC,"Richmond, VA",VA,Virginia,US,United States,38,DEN,"Denver, CO",...,United States,82,2019,2,6,3,0.0,1482.0,F,RICDEN
4,Southwest Airlines Co.,RIC,"Richmond, VA",VA,Virginia,US,United States,38,LAS,"Las Vegas, NV",...,United States,85,2019,2,6,5,0.0,2086.0,F,RICLAS
5,Southwest Airlines Co.,RNO,"Reno, NV",NV,Nevada,US,United States,85,PHL,"Philadelphia, PA",...,United States,23,2019,2,6,5,0.0,2348.0,F,RNOPHL
6,Southwest Airlines Co.,RSW,"Fort Myers, FL",FL,Florida,US,United States,33,HOU,"Houston, TX",...,United States,74,2019,2,6,2,0.0,853.0,F,RSWHOU
7,Southwest Airlines Co.,LAX,"Los Angeles, CA",CA,California,US,United States,91,SJU,"San Juan, PR",...,United States,3,2019,3,7,7,0.0,3386.0,F,LAXSJU
8,Southwest Airlines Co.,MKE,"Milwaukee, WI",WI,Wisconsin,US,United States,45,BOI,"Boise, ID",...,United States,83,2019,3,7,3,0.0,1423.0,F,MKEBOI
9,Southwest Airlines Co.,OAK,"Oakland, CA",CA,California,US,United States,91,AMA,"Amarillo, TX",...,United States,74,2019,3,7,3,0.0,1153.0,F,OAKAMA


In [9]:
passengers_by_flight = passdata.groupby(['YEAR','flight'], as_index = False).agg({'PASSENGERS':'sum'})
passengers_by_flight.sort_values(by=['YEAR','PASSENGERS'], ascending=False)

flights2020 = pd.DataFrame(passengers_by_flight.loc[passengers_by_flight['YEAR']==2020,:])
top20flights_2020 = flights2020.sort_values(by=['YEAR','PASSENGERS'], ascending=False).head(20)

flights2019 = pd.DataFrame(passengers_by_flight.loc[passengers_by_flight['YEAR']==2019,:])
top20flights_2019 = flights2019.sort_values(by=['YEAR','PASSENGERS'], ascending=False).head(20)



In [16]:
df = len(top20flights_2020)-1
q=0.95

top20flights_2020['expected'] = round(top20flights_2020['PASSENGERS'].mean(),0)

print(df,round(top20flights_2020['PASSENGERS'].mean(),0))

#critical value
critical_value = st.chi2.ppf(q=q,df=df)
print(f'the critical value is {critical_value}')

st.chisquare(top20flights_2020['PASSENGERS'],top20flights_2020['expected'])


19 349777.0
the critical value is 30.14352720564616


Power_divergenceResult(statistic=50103.73374750198, pvalue=0.0)

In [17]:
df = len(top20flights_2019)-1
q=0.95

top20flights_2019['expected'] = round(top20flights_2019['PASSENGERS'].mean(),0)

print(df,round(top20flights_2019['PASSENGERS'].mean(),0))

#critical value
critical_value = st.chi2.ppf(q=q,df=df)
print(f'the critical value is {critical_value}')

st.chisquare(top20flights_2019['PASSENGERS'],top20flights_2019['expected'])


19 870701.0
the critical value is 30.14352720564616


Power_divergenceResult(statistic=262415.12716765003, pvalue=0.0)

In [24]:
#passengers by state
passengers_by_state = passdata.groupby(['YEAR','ORIGIN_STATE_NM'], as_index = False).agg({'PASSENGERS':'sum'})
passengers_by_state.sort_values(by=['YEAR','PASSENGERS'], ascending=False).head(20)

stateflights2020 = pd.DataFrame(passengers_by_state.loc[passengers_by_state['YEAR']==2020,:])
top20states_2020 = stateflights2020.sort_values(by=['YEAR','PASSENGERS'], ascending=False).head(20)

stateflights2019 = pd.DataFrame(passengers_by_state.loc[passengers_by_state['YEAR']==2019,:])
top20states_2019 = stateflights2019.sort_values(by=['YEAR','PASSENGERS'], ascending=False).head(20)

In [25]:
df = len(top20states_2020)-1
q=0.95

top20states_2020['expected'] = round(top20states_2020['PASSENGERS'].mean(),0)

print(df,round(top20states_2020['PASSENGERS'].mean(),0))

#critical value
critical_value = st.chi2.ppf(q=q,df=df)
print(f'the critical value is {critical_value}')

st.chisquare(top20states_2020['PASSENGERS'],top20states_2020['expected'])


19 9853062.0
the critical value is 30.14352720564616


Power_divergenceResult(statistic=110591873.69335493, pvalue=0.0)

In [27]:
df = len(top20states_2019)-1
q=0.95

top20states_2019['expected'] = round(top20states_2019['PASSENGERS'].mean(),0)

print(df,round(top20states_2019['PASSENGERS'].mean(),0))

#critical value
critical_value = st.chi2.ppf(q=q,df=df)
print(f'the critical value is {critical_value}')

st.chisquare(top20states_2019['PASSENGERS'],top20states_2019['expected'])


19 23374192.0
the critical value is 30.14352720564616


Power_divergenceResult(statistic=253779885.33529663, pvalue=0.0)

In [26]:
#round trip vs one way
top20states_2020

,YEAR,ORIGIN_STATE_NM,PASSENGERS,expected
72,2020,Florida,27817533.0,9853062.0
69,2020,California,26581043.0,9853062.0
115,2020,Texas,24084987.0,9853062.0
73,2020,Georgia,12968892.0,9853062.0
76,2020,Illinois,11919391.0,9853062.0
97,2020,New York,11083475.0,9853062.0
70,2020,Colorado,9622023.0,9853062.0
99,2020,North Carolina,9579883.0,9853062.0
66,2020,Arizona,7400084.0,9853062.0
92,2020,Nevada,6911985.0,9853062.0
